# Performance of sampled the state space of all the possible TensorFlow==2.1.0

The purpose of this notebook is to show performance of all possible combinations of the software stack for TensorFlow==2.1.0. These results obtained using Amun service explain some of the knowledge stored into Thoth that is used to provide Advises on AI software stacks.

## Inputs for Dataset created using Amun

All data have been gathered using [Amun Service](https://github.com/thoth-station/amun-api) and [Performance Indicators](https://github.com/thoth-station/performance) evaluated by Thoth Team.


## TensorFlow builds

Tensorflow builds have been created considering combinations of the following parameters:

**Software stacks and native dependencies**

All inspections use a combination of all stacks from the dependencies of TensorFlow in version 2.1.0. 


  * `upstream TensorFlow` - `tensorflow==2.1.0` available on PyPI (inspections prefixed with `tf`)

**OS images**

  * `rhel-8` 

**Python Interpreters**

  * `3.6` 
  
**Hardware**

No node pinning used, any hardware available on OCP is used. No GPU was used. 
Analysis across inspection run will show which hardware have been identified.

`Number of CPUs` used to run is selected a priori as input to Amun: 1

## Performance indicators
Performance Indicators (PI) used for performance analysis:

  * [matrix multiplication](https://github.com/thoth-station/performance/blob/master/tensorflow/matmul.py)

Each performance indicator was run `1 times` per inspection run (`batch size == 1`), performance indicators reported median of inspections to be further compared.

## Dataset content

Inspection specification, build logs, job logs, hardware information of the node where the performance indicator was run and the actual inspection job result are included in the dataset.

No buildtime errors spotted with the tested stack.

There are some runtime errors spotted with specific stack.


## Analysis

Results of performance are shown in terms of Elapsed time [ms].

The analysis performed in this notebook are defined as:

- Performance analysis across different Tf stacks (Python packages) (fixed Hardware, OS image, Python Interpreter, number of CPUs)


## Set environment variables to access the datasets on Ceph

For more detail on the Operate First Ceph public bucket used here, visit https://github.com/operate-first/apps/blob/master/docs/odh/trino/access_public_bucket.md

In [ ]:
%env THOTH_CEPH_KEY_ID=LLEzCoxu7pvjzO4inoL8
%env THOTH_CEPH_SECRET_KEY=1HnDVoIS2jt3h3xEpgeQlCX5+FeOUH0wOrvWVvZP
%env THOTH_CEPH_BUCKET_PREFIX=thoth
%env THOTH_S3_ENDPOINT_URL=https://s3-openshift-storage.apps.smaug.na.operate-first.cloud
%env THOTH_CEPH_BUCKET=opf-datacatalog
%env THOTH_DEPLOYMENT_NAME=datasets

In [ ]:
from thoth.report_processing.components.inspection import AmunInspections
from thoth.report_processing.components.inspection import AmunInspectionsSummary
from thoth.report_processing.components.inspection import AmunInspectionsStatistics
from thoth.report_processing.components.inspection import AmunInspectionsFailedSummary

inspection = AmunInspections()
inspection_runs_summary = AmunInspectionsSummary()
inspection_statistics = AmunInspectionsStatistics()
inspections_failed_sumary = AmunInspectionsFailedSummary()

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1500)
pd.options.plotting.backend = "plotly"  # Convert to matplotlib

In [ ]:
inspections_identifiers = ["tf-dm-rw"]

In [ ]:
inspection_runs = inspection.aggregate_thoth_inspections_results(
    inspections_identifiers=inspections_identifiers
)

In [ ]:
processed_inspection_runs, failed_inspection_runs = inspection.process_inspection_runs(
    inspection_runs
)

In [ ]:
inspections_df = inspection.create_inspections_dataframe(
    processed_inspection_runs=processed_inspection_runs,
    include_statistics=True
)

In [ ]:
inspections_failed_df = inspection.create_inspections_dataframe(
    processed_inspection_runs=failed_inspection_runs
)

# Failed Inspections

In [ ]:
inspections_failed_df.head()

In [ ]:
inspections_failed_df['stderr'][0]

## Create inspection results summary

In [ ]:
report_results_failed, _ = inspection_runs_summary.produce_summary_report(inspections_df=inspections_failed_df)

### Hardware

In [ ]:
report_results_failed["hardware"]['platform'].head()

In [ ]:
report_results_failed["hardware"]['processor']

In [ ]:
report_results_failed["hardware"]['flags']

In [ ]:
report_results_failed["hardware"]['ncpus']

In [ ]:
report_results_failed["hardware"]['info']

### Operating System

In [ ]:
report_results_failed["base_image"]['base_image']

In [ ]:
report_results_failed["base_image"]['number_cpus_run']

### Performance Indicators

In [ ]:
report_results_failed["pi"]['pi']

### Software Stack

In [ ]:
report_results_failed["software_stack"]['requirements_locked'].head()

In [ ]:
python_packages_dataframe_failed, _ = inspection.create_python_package_df(inspections_df=inspections_failed_df)
python_packages_dataframe_failed.head()

# Successfull inspections

In [ ]:
inspections_df.head()

## Create inspection results summary

In [ ]:
report_results, _ = inspection_runs_summary.produce_summary_report(inspections_df=inspections_df)

### Hardware

In [ ]:
report_results["hardware"]['platform'].head()

In [ ]:
report_results["hardware"]['processor']

In [ ]:
report_results["hardware"]['flags']

In [ ]:
report_results["hardware"]['ncpus']

In [ ]:
report_results["hardware"]['info']

### Operating System

In [ ]:
report_results["base_image"]['base_image']

In [ ]:
report_results["base_image"]['number_cpus_run']

### Performance Indicators

In [ ]:
report_results["pi"]['pi']

### Software Stack

In [ ]:
report_results["software_stack"]['requirements_locked'].head()

In [ ]:
python_packages_dataframe, _ = inspection.create_python_package_df(inspections_df=inspections_df)
python_packages_dataframe.head()

In [ ]:
final_dataframe = inspection.create_final_dataframe(
    inspections_df=inspections_df,
    include_statistics=True
)
final_dataframe.head()

## Plot results

In [ ]:
import os

from typing import List

import pandas as pd
import plotly.graph_objects as go

from typing import Optional, Dict

pd.options.plotting.backend = "plotly"

from kaleido.scopes.plotly import PlotlyScope
scope = PlotlyScope()

if not os.path.exists("images"):
    os.mkdir("images")

_PERFORMANCE_QUANTITY = ["elapsed_time", "rate"]

_PERFORMANCE_QUANTITY_MAP = {"elapsed_time": "Elapsed Time [ms]", "rate": "Rate [GFLOPS]"}


class AmunInspectionsVisualization:
    """Class of methods used to create statistics from Amun Inspections Runs."""

    @staticmethod
    def create_inspection_3d_plot(
        plot_df: pd.DataFrame,
        varying_package: str,
        title_plot: str,
        quantity: Optional[str] = 'elapsed_time',
        image_name: Optional[str] = None,
    ):
        """Create inspection performance parameters plot in 3D.

        :param final_inspections_df: df for plots provided by `create_final_dataframe` or its subset.
        :param 
        """
        if quantity not in _PERFORMANCE_QUANTITY:
            _LOGGER.info(f"Only {_PERFORMANCE_QUANTITY} are accepted as quantity")
            return

        x_vector = [x[0] for x in plot_df[["solver_string"]].values]

        integer_y_encoded = [y[0] for y in plot_df[["sws_hash_id_encoded"]].values]

        z_vector = [z[0] for z in plot_df[[quantity]].values]

        trace1 = go.Scatter3d(
            x=x_vector,
            y=integer_y_encoded,
            z=z_vector,
            mode="markers",
            hovertext=[yc[0] for yc in plot_df[["sws_string"]].values],
            hoverinfo="text",
            marker=dict(
                size=8,
                color=z_vector,  # set color to an array/list of desired values
                colorscale="Viridis",  # choose a colorscale
                opacity=0.8,
                showscale=True,
            ),
        )

        data = [trace1]

        annotations = []
        c = 0

        for (x, y, z) in zip(x_vector, integer_y_encoded, z_vector):
            annotations.append(
                dict(
                    showarrow=False,
                    x=x,
                    y=y,
                    z=z,
                    text="".join(plot_df[varying_package].values[c]),
                    xanchor="left",
                    xshift=15,
                    opacity=0.7,
                )
            )
            c += 1

        margin = {"l": 200, "r": 100, "b": 100, "t": 100}

        layout = go.Layout(
            title=title_plot,
#             margin=margin,
            scene=dict(
                xaxis=dict(
                    title="Runtime Environment",
                    backgroundcolor="rgb(200, 200, 230)",
                    gridcolor="white",
                    showbackground=True,
                    zerolinecolor="white",
                    ),
                yaxis=dict(title="Software Stack ID",
                    backgroundcolor="rgb(230, 200,230)",
                    gridcolor="white",
                    showbackground=True,
                    zerolinecolor="white",
                    ),
                zaxis=dict(title=_PERFORMANCE_QUANTITY_MAP[quantity],
                    backgroundcolor="rgb(230, 230,200)",
                    gridcolor="white",
                    showbackground=True,
                    zerolinecolor="white"),
#                 annotations=annotations
            ),
            showlegend=True,
            legend=dict(orientation="h"),
        )
        fig = go.Figure(data=data, layout=layout)

        if not image_name:
            image_name = title_plot

        with open(f"images/{image_name}.png", "wb") as f:
            f.write(scope.transform(fig, format="png"))

        return fig
    
    
    @staticmethod
    def create_inspection_2d_plot(
        plot_df: pd.DataFrame,
        varying_package: str,
        title_plot: str,
        quantity: Optional[str] = 'elapsed_time',
        have_annotations: bool = False,
        image_name: Optional[str] = None,
    ):
        """Create inspection performance parameters plot in 2D.

        :param final_inspections_df: df for plots provided by `create_final_dataframe` or its subset.
        """
        integer_y_encoded = [y[0] for y in plot_df[["sws_hash_id_encoded"]].values]

        data = []
        annotations = []
        colour_counter = 0
        distance: float = 0
        name_component = varying_package

        subset_df = plot_df[plot_df["pi_component"] == varying_package]
        z_vector = [z[0] for z in subset_df[[quantity]].values]

        trace = go.Scatter(
            x=integer_y_encoded,
            y=z_vector,
            mode="markers",
            hovertext=[y[0] for y in subset_df[["sws_string"]].values],
            hoverinfo="text",
            marker=dict(
                size=12,
                color=z_vector,  # set color to an array/list of desired values
#                 colorscale=color_scales[colour_counter],  # choose a colorscale
                opacity=0.8,
                showscale=True,
                colorbar={"x": 1 + distance},
            ),
            name=f"solver=={plot_df['solver_string'].unique()[0]}",
        )

        data.append(trace)
        colour_counter += 1
        distance += 0.2

        layout = go.Layout(
            title=title_plot,
            xaxis=dict(title="Software Stack ID integer encoded"),
            yaxis=dict(title=_PERFORMANCE_QUANTITY_MAP[quantity]),
            showlegend=True,
            legend=dict(orientation="h", y=-0.3, yanchor="top"),
        )
        fig = go.Figure(data=data, layout=layout)

        if not image_name:
            image_name = title_plot

        with open(f"images/{image_name}.png", "wb") as f:
            f.write(scope.transform(fig, format="png"))

        return fig

In [ ]:
AmunInspectionsVisualization.create_inspection_3d_plot(plot_df=final_dataframe, varying_package="tensorflow", title_plot="TF==2.1.0 performance (3D plot)")

In [ ]:
AmunInspectionsVisualization.create_inspection_2d_plot(plot_df=final_dataframe, varying_package="tensorflow", title_plot="TF==2.1.0 performance (2D plot)")